<center> 

# Bioinformática Avanzada 2025

## TP Nº1: Introducción a Biopython
Daniela Rodríguez Golpe


### **Programa detector de ORFs a partir de una secuencia de ADN humano**

- El programa recibe una secuencia de ADN brindada por el usuario y permite filtrar ORFs con un tamaño mínimo (input)

- El programa muestra todos los ORFs encontrados, destacando el más largo de ellos y si es o no probable que sea un gen/exon (output)


- *Secuencia real de prueba (control +)*: 
Gen de la Beta-Globina Humana (HBB) (inicio ATG, terminación con codon STOP y codifica una proteína funcional):

ATGGTGCACCTGACTCCTGAGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGTTGGTATCAAGGTTACAAGACAGGTTTAAGGAGACCAATAGAAACTGGGCATGTGGAGACAGAGAAGACTCTTGGGTTTCTGATAGGCACTGACTCTCTCTGCCTATTGGTCTATTTTGGCAGGTTTTGAGTCCTTTGGGGATCTGTCCACTCCTGATGCTGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACGTGGATCCTGAGAACTTCAGGGTGAGTCTATGGGACGCTTGATGTCTCTGTCTCCACAGGAGTCAGGTGCAGGCGGAGGAGACGTCTG

- *Secuencia aleatoria de prueba (control -)*: 
ATGAAATGAAAAATAGATGTTTTAAATGCCCTGATGTTTGGGATGAATAG

Archivos externos necesarios: human_500_nonredundant.fasta

Contiene 500 proteinas humanas no redundantes a partir de las cuales se obtuvo la distribución de las frecuencias de aa de proteínas humanas reales obtenidas de Uniprot

In [1]:
#Flujo principal del programa resumido
def orf_flowchart_summary():
    print("START → Analizar secuencia de ADN? → Ingresar secuencia y validar (A,T,C,G)")
    print("Generar 6 marcos → Buscar ORFs (ATG→STOP) → Filtrar por tamaño mínimo")
    print("Calcular frecuencias, RMSE y z-scores → Determinar ORFs probables → Mostrar y guardar resultados → END")

# Ejecutar
orf_flowchart_summary()

START → Analizar secuencia de ADN? → Ingresar secuencia y validar (A,T,C,G)
Generar 6 marcos → Buscar ORFs (ATG→STOP) → Filtrar por tamaño mínimo
Calcular frecuencias, RMSE y z-scores → Determinar ORFs probables → Mostrar y guardar resultados → END


In [32]:

import numpy as np
import random
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from collections import Counter
import math
import json


In [33]:
# ----- Bloque I: Valida que las secuencias dadas por el usuario sean de ADN -----
def validation_seq(seq, type):
    if type == "DNA":
        valid_bases = set("ATCG")
        return all(base in valid_bases for base in seq)
    else:
        return False


# ----- Bloque II: Marcos de lectura y búsqueda de ORFs ----
# --- Bloque 2A: Obtención de marcos de lectura---
def get_reading_frames(seq_str):
    """
    Obtiene los 6 marcos de lectura (3 directos y 3 reversos) como listas de codones.
    seq_str: secuencia de ADN como string. Retorna: lista de listas de codones por marco
    """
    seq = Seq(seq_str)
    rev_seq = seq.reverse_complement()
    frames = []

    # 3 marcos directos
    for i in range(3):
        sub_seq = seq[i:]
        codons = [str(sub_seq[j:j+3]) for j in range(0, len(sub_seq)-2, 3)]
        frames.append(codons)
    # 3 marcos reversos
    for i in range(3):
        sub_seq = rev_seq[i:]
        codons = [str(sub_seq[j:j+3]) for j in range(0, len(sub_seq)-2, 3)]
        frames.append(codons)
    return frames


# --- Bloque 2B: Búsqueda de ORFs ---
def find_orfs(frames):
    """
    Encuentra todos los ORFs en los marcos de lectura dados.
    frames: lista de listas de codones (6 marcos de lectura)
    Retorna una lista de diccionarios con info del ORF
    """
    start_codon = "ATG"
    stop_codons = {"TAA", "TAG", "TGA"}
    orfs = []

    for frame_index, codon_list in enumerate(frames):
        i = 0
        while i < len(codon_list):
            if codon_list[i] == start_codon:
                orf_codons = [codon_list[i]]
                for j in range(i + 1, len(codon_list)):
                    codon = codon_list[j]
                    orf_codons.append(codon)
                    if codon in stop_codons:
                        nt_seq = ''.join(orf_codons)
                        aa_seq = str(Seq(nt_seq).translate(to_stop=True))  # Se traduce hasta el primer STOP
                        orfs.append({
                            "frame": frame_index + 1,
                            "direction": "Direct" if frame_index < 3 else "Reverse",
                            "start": i * 3,
                            "end": (j + 1) * 3,
                            "length_nt": (j - i + 1) * 3,
                            "sequence_nt": nt_seq,
                            "sequence_aa": aa_seq
                        })
                        break
                i = j + 1
            else:
                i += 1
    return orfs


# ---- Bloque III: Comparación de distribuciones ----
# --- Bloque 2A: Determinación del largo y distribución de aa de cada ORF ---
def analyze_orfs(orfs):
    """
    Calcula longitud en aa y distribución % de aa para cada ORF.
    Modifica lista de ORFs agregando 'length_aa' y 'aa_freq_percent'.
    """
    for orf in orfs:
        aa_seq = orf["sequence_aa"]
        length_aa = len(aa_seq)
        freq = Counter(aa_seq)
        total = sum(freq.values())
        freq_percent = {aa: (count / total) * 100 for aa, count in freq.items()}
        orf["length_aa"] = length_aa
        orf["aa_freq_percent"] = freq_percent
    return orfs

#--- Bloque 2B: filtro de ORFs menores a los aa que decida el usuario
def filter_orfs_by_length(orfs, min_aa_length):
    """
    Filtra ORFs que tengan menos de min_aa_length aminoácidos (sin contar el STOP).
    """
    return [orf for orf in orfs if len(orf['sequence_aa']) >= min_aa_length]

#--- Bloque 2C: Calculo de RMSE ---
def rmse_distribution(freq1, freq2):
    """
    Calcula RMSE entre dos distribuciones de aminoácidos.
    freq1 y freq2: diccionarios {aa: frecuencia} en proporciones (0-1).
    """
    amino_acids = sorted(set(freq1.keys()) | set(freq2.keys()))
    squared_errors = []
    for aa in amino_acids:
        f1 = freq1.get(aa, 0)
        f2 = freq2.get(aa, 0)
        squared_errors.append((f1 - f2) ** 2)
    mse = sum(squared_errors) / len(amino_acids)
    return math.sqrt(mse)

def z_score(value, mean, std):
    if std == 0:
        return 0
    return (value - mean) / std

# --- Bloque 2D: Comparación con distribuciones de proteinas reales y proteinas generadas al azar ---
def compare_orfs_to_real_and_random(orfs, real_freq, random_freq, real_rmse_dist, rand_rmse_dist):
    """
    Compara cada ORF con distribuciones de referencia usando z-score y RMSE.
    """
    real_mean = np.mean(real_rmse_dist)
    real_std = np.std(real_rmse_dist)
    rand_mean = np.mean(rand_rmse_dist)
    rand_std = np.std(rand_rmse_dist)

    for orf in orfs:
        orf_freq = orf.get("aa_freq_percent", {})
        orf_freq_prop = {aa: val / 100 for aa, val in orf_freq.items()}

        rmse_real = rmse_distribution(orf_freq_prop, real_freq)
        rmse_random = rmse_distribution(orf_freq_prop, random_freq)

        z_real = z_score(rmse_real, real_mean, real_std)
        z_random = z_score(rmse_real, rand_mean, rand_std)

        orf["rmse_to_real"] = rmse_real
        orf["rmse_to_random"] = rmse_random
        orf["z_rmse_real"] = z_real
        orf["z_rmse_random"] = z_random
        orf["is_likely_gene"] = abs(z_real) < abs(z_random)
    return orfs

def real_protein_analysis(file_name, output_json="frecuencias_aa.json"):
    """
    Lee archivo FASTA con proteínas reales, calcula frecuencia relativa de aa,
    guarda en JSON y retorna diccionario de frecuencias relativas.
    """
    print("\nIniciando análisis de aminoácidos reales:")
    total_aa_counts = Counter()
    try:
        with open(file_name, "r") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                sequence = str(record.seq).upper()
                if all(base in "ARNDCQEGHILKMFPSTWYV" for base in sequence):
                    total_aa_counts.update(sequence)
                else:
                    print(f"Secuencia {record.id} no válida, se omite.")
        if total_aa_counts:
            total = sum(total_aa_counts.values())
            freq_dict = {aa: total_aa_counts[aa] / total for aa in sorted(total_aa_counts)}
            with open(output_json, "w") as f:
                json.dump(freq_dict, f, indent=4)
            print(f"Frecuencias reales guardadas en {output_json}")
            return freq_dict
        else:
            print("No se encontraron secuencias válidas.")
            return {}
    except FileNotFoundError:
        print(f"Archivo '{file_name}' no encontrado.")
        return {}
    except Exception as e:
        print(f"Error inesperado: {e}")
        return {}

def generate_random_proteins(n=500, min_len=100, max_len=800):
    """
    Genera n secuencias de proteínas aleatorias con longitudes entre min_len y max_len.
    Retorna la distribución global de aminoácidos en proporciones.
    """
    amino_acids = list("ARNDCQEGHILKMFPSTWYV")
    total_counts = Counter()

    for i in range(n):
        length = random.randint(min_len, max_len)
        seq = random.choices(amino_acids, k=length)
        total_counts.update(seq)
    total = sum(total_counts.values())
    freq_prop = {aa: count / total for aa, count in total_counts.items()}
    return freq_prop

def simulate_rmse_distributions(real_freq, random_freq, n=1000, min_len=100, max_len=800):
    """
    Simula distribuciones de RMSE comparando secuencias aleatorias contra distribuciones reales y aleatorias.
    Devuelve dos listas: RMSE contra real y RMSE contra aleatoria.
    """
    amino_acids = list("ARNDCQEGHILKMFPSTWYV")
    real_rmse = []
    rand_rmse = []

    for _ in range(n):
        # Genera secuencia aleatoria
        length = random.randint(min_len, max_len)
        seq = random.choices(amino_acids, k=length)
        count = Counter(seq)
        total = sum(count.values())
        freq_prop = {aa: count[aa] / total for aa in amino_acids}

        # RMSE con distribuciones reales y aleatorias
        real_rmse.append(rmse_distribution(freq_prop, real_freq))
        rand_rmse.append(rmse_distribution(freq_prop, random_freq))
    return real_rmse, rand_rmse

def save_orfs_to_json(orfs, filename="orfs_resultados.json"):
    """
    Guarda la lista de ORFs en un archivo JSON.
    Convierte valores no serializables como numpy.bool_ o numpy.float_ a tipos nativos de Python.
    """
    try:
        def convert(o):
            if isinstance(o, (np.bool_, np.bool8)):
                return bool(o)
            if isinstance(o, (np.float32, np.float64)):
                return float(o)
            return o

        with open(filename, "w") as f:
            json.dump(orfs, f, indent=4, default=convert)
        print(f"\nResultados guardados en '{filename}'")
    except Exception as e:
        print(f"Error al guardar los resultados: {e}")

# ---- Bloque IV: Programa principal ----
def run():
    menu="""
    Welcome to the ORFs detector. Do you have a sequence to analyze?
    1- Yes, a DNA sequence
    2- No
    Pick up an option: """

    try:
        option = int(input(menu))
        if option == 1:
            seq_ = str(''.join(input("Paste your sequence: ").split()).upper())
            z = int(input("¿Qué tamaño de ORFs desea filtrar? igrese un número (sólo se tendrán en cuenta ORFs a partir de ese valor): "))
            if validation_seq(seq_, "DNA"):
                print("Valid DNA sequence.")
                # Llama al Bloque 2A: obtener marcos de lectura
                frames = get_reading_frames(seq_)
                # Llama al Bloque 2B: buscar ORFs
                orfs = find_orfs(frames)
                # Filtro de ORFs pequeños
                orfs = filter_orfs_by_length(orfs, min_aa_length=z)
                # Analiza proteínas reales y obtiene una distribución
                real_freq = real_protein_analysis("human_500_nonredundant.fasta")
                # Genera distribución aleatoria de proteínas
                random_freq = generate_random_proteins()
                # Simulación de distribuciones de RMSE
                real_rmse_dist, rand_rmse_dist = simulate_rmse_distributions(real_freq, random_freq)
                # Analiza los ORFs
                orfs = analyze_orfs(orfs)
                #Etiqueta el ORF más largo
                if orfs:
                    longest_length = max(orf['length_nt'] for orf in orfs)
                    for orf in orfs:
                        orf['is_longest'] = (orf['length_nt'] == longest_length)
                if not (real_freq and random_freq):
                    print("Error: No se pudieron obtener distribuciones reales o aleatorias.")
                    return # Termina el programa si faltan datos clave
                # Compara ORFs con distribuciones reales y aleatorias
                orfs = compare_orfs_to_real_and_random(orfs, real_freq, random_freq, real_rmse_dist, rand_rmse_dist)
                if orfs:
                    print(f"\n{len(orfs)} ORF(s) found:\n")
                    for i, orf in enumerate(orfs, 1):
                        longest_tag = " (Longest ORF Detected)" if orf.get('is_longest') else ""
                        print(f"ORF {i}{longest_tag}:")
                        print(f"  Frame: {orf['frame']} ({orf['direction']})")
                        print(f"  Position: nt {orf['start']} – {orf['end']}")
                        print(f"  Length: {orf['length_nt']} nucleotides / {orf['length_aa']} aa")
                        print(f"  RMSE to real: {orf['rmse_to_real']:.4f}")
                        print(f"  RMSE to random: {orf['rmse_to_random']:.4f}")
                        print(f"  Likely gene/exon: {'YES' if orf['is_likely_gene'] else 'NO'}")
                        print(f"  Amino acid sequence: {orf['sequence_aa']}")
                        print("-" * 50)
                        print(f"  Z-score to real: {orf['z_rmse_real']:.2f}")
                        print(f"  Z-score to random: {orf['z_rmse_random']:.2f}")
                    save_orfs_to_json(orfs)
                else:
                    print("No ORFs found in the sequence.")
            else:
                print("Error: The sequence is not a valid DNA sequence.")
        elif option == 2:
                print("Thank you for use the detector of ORFs program, see you next time")
        else:
            print("Pick up a correct option (1 or 2)")
    except ValueError:
        print("Invalid input. Please enter a number.")


if __name__ == '__main__':
    run()


Valid DNA sequence.

Iniciando análisis de aminoácidos reales:
Frecuencias reales guardadas en frecuencias_aa.json

7 ORF(s) found:

ORF 1 (Longest ORF Detected):
  Frame: 1 (Direct)
  Position: nt 0 – 186
  Length: 186 nucleotides / 61 aa
  RMSE to real: 0.0278
  RMSE to random: 0.0342
  Likely gene/exon: YES
  Amino acid sequence: MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLVSRLQDRFKETNRNWACGDREDSWVSDRH
--------------------------------------------------
  Z-score to real: 1.04
  Z-score to random: 4.20
ORF 2:
  Frame: 1 (Direct)
  Position: nt 249 – 276
  Length: 27 nucleotides / 8 aa
  RMSE to real: 0.0875
  RMSE to random: 0.0918
  Likely gene/exon: NO
  Amino acid sequence: MLLWATLR
--------------------------------------------------
  Z-score to real: 20.07
  Z-score to random: 19.44
ORF 3:
  Frame: 1 (Direct)
  Position: nt 282 – 360
  Length: 78 nucleotides / 25 aa
  RMSE to real: 0.0446
  RMSE to random: 0.0470
  Likely gene/exon: YES
  Amino acid sequence: MARKCSVPLVMAWLTWTTSRAPLPH
------

C:\Users\Usuario\AppData\Local\Temp\ipykernel_712\1048837680.py:220: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if isinstance(o, (np.bool_, np.bool8)):
